### This notebook searches the requests data for instances of device identifiers

In [1]:
%pylab inline
from IPython.display import display, clear_output
import sys
import time
import pickle
import cgi
import urlparse
import utils

Populating the interactive namespace from numpy and matplotlib


### The cell below can be used to explore new values for various keys

In [2]:
keyword = "deviceID"
expression = "request.scheme == 'http' and ('%s' in request.path or '%s' in request.content)" % (keyword, keyword)
request_data = utils.fetch_filtered_requests(expression, limit=10)
utils.print_requests(request_data)

===
Run ID: 1430076923
Package: com.xe.currency
Scheme: http
Host: a.tribalfusion.com
Path: /insights/impression?client=589423&campaign=(4777775)&mediaSource=appsnack&creative=320x50&event=MedLoad&ord=1637435225&custom1=dp:deviceID:209485;osID:210310;browserID:180036&custom2=buyID:8548812;mediaID:4777775;ord:1637435225;adspaceId:304041;sp:AdMob
Content: 
===
Run ID: 1430076923
Package: com.xe.currency
Scheme: http
Host: a.tribalfusion.com
Path: /z/insights/impression?client=589423&campaign=(4777775)&mediaSource=appsnack&creative=320x50&event=MedLoad&ord=1637435225&custom1=dp:deviceID:209485;osID:210310;browserID:180036&custom2=buyID:8548812;mediaID:4777775;ord:1637435225;adspaceId:304041;sp:AdMob
Content: 
===
Run ID: 1430046674
Package: lunosoftware.mlbscores
Scheme: http
Host: www.lunosoftware.com
Path: /sportsdata/SportsDataService.svc/startupValues
Content: platform=Android&deviceVersion=5.0.1&appID=android.mlbscores&appVersion=1.3.3&deviceID=AC46E412-7378-4416-AB44-AF5AD6837C30
==

In [3]:
some_identifiers = {
    "deviceId_1" : "dcd27149-60cb-49d7-bd70-775e01b945a8",
    "deviceID_2" : "352584061481484",
    "sha1_udid" : "3f4c353c7837a950dddae386fd8912bb4a27db5b",
    "hudid" : "2988567d39e4d8560db06871a9657cec3fd10489",
    "uuid_1" : "6af7d1af821cda51ff9463d471a69de7",
    "uuid_2" : "2512c59bc73780cbe640025a039080bf",
    "uuid_3" : "352584061481484:f8a9d08615fb:000",
    "androidID" : "ec089e6192d63b18"
}

### That seemed to be a useful exercise. Let's try to discover interesting keys and values programatically so we can find them being used across apps and across hosts

### Any automatic system needs to start by gleaning keys that are likely identifiers. We can do this by collecting all the keys that have the phrase 'id' in them

In [4]:
total_run_count = utils.runs.count()
print "Total runs: %s" % total_run_count
all_keys = {} # all_key : list of (package_title, host, value)

counter = 0
for run in utils.runs.find():
    clear_output(wait=True)
    print(counter)
    sys.stdout.flush()

    package_title = run["package_title"]
    run_id = run["run_id"]
    requests = [pickle.loads(x) for x in run["requests"]]

    for request in requests:
        scheme = request.scheme
        host = request.host
        path = request.path
        content = request.content

        url = scheme + "://" + host + path
        url_data_dict = cgi.parse_qs(urlparse.urlparse(url).query)

        # go one level deeper for doubleclick data
        temp_url_data_dict = url_data_dict.copy()
        for key, value in url_data_dict.iteritems():
            try:
                if len(value) == 1:
                    parsed = cgi.parse_qs(value[0])
                    if parsed:
                        del temp_url_data_dict[key]
                        temp_url_data_dict.update(parsed)
            except:
                continue
        url_data_dict = temp_url_data_dict

        content_data_dict = cgi.parse_qs(content)

        url_data_tosave = {}
        content_data_tosave = {}
        
        if url_data_dict or content_data_dict:
            try:
                url_data_tosave = {unicode(key, "latin-1").encode('ascii', 'replace').replace('\x00', '').replace(".","").replace("$",""): [unicode(x, "latin-1").encode('ascii', 'replace') for x in value] for (key, value) in url_data_dict.items()}
            except:
                pass

            try:
                content_data_tosave = {unicode(key, "latin-1").encode('ascii', 'replace').replace('\x00', '').replace(".","").replace("$",""): [unicode(x, "latin-1").encode('ascii', 'replace') for x in value] for (key, value) in content_data_dict.items()}
            except:
                pass

        key_data_dicts = [url_data_tosave, content_data_tosave]
        
        for data_dict in key_data_dicts:
            for key in data_dict:
                if key in all_keys:
                    all_keys[key].append((package_title, host, data_dict[key]))
                else:
                    all_keys[key] = [((package_title, host, data_dict[key]))]
                    
    counter += 1

2424


In [5]:
id_keys = {} # id_key : list of (package_title, host, value)

for key in all_keys:
    if "id" not in key.lower() \
        or "\\" in key \
        or "/" in key \
        or len(key) > 20:
        continue
    id_keys[key] = all_keys[key]

### Here's what some of the keys in `id_keys` look like

In [6]:
print id_keys.keys()[0:100]

['insideApp', 'adnxs_uid', 'udId', 'fb_id', 'dataProviderId', 'displayWidth', 'Q_CID', 'upsid', 'pidt', 'cmMID', 'apptimize_test_id', 'bidder_name', 'epomUUID', 'RegionId', 'city_id', ' "><div id', 'adv_reference_id', 'pageId', 'wbmdid', 'void', 'api[uuid]', 'marketplaceId', 'bkrid', 'd(id12)', 'responseid', 'x_provider', 'osdevid', 'clickID', 'AG_APP_ID', 'pubid', 'c_synch_userid', 'AndroidTest', 'appnxsid', 'pub_id', 'registration_id', 'asid', 'os_id', 'PartnerId', 'aAppID', 'androidVersion', 'u-appBId', 'tmpageid', 'chanId', 'atlasId', 'timeZoneId', 'd_orgid', 'liftoff_user_id', 'goal_id', 'vehicleids', 'partner_uid', 'entity_id', 'placement_id', 'unid', 'ymid', 'video_length', 'au_id', 'towerID', 'auid', 'oldRegId', 'incidentsource', 'send_id', 'countryid', 'subpanelid', 'offer_id', 'ltid', 'category_tid', 'FrontDoorAffiliateId', 'omSuiteId', 'icctm_ht_aid', 'AdvertiserID', '_RM_HTML_ID_', 'metricId', '_pid', ' DPIdentifier', 'utvid', 'trackid', 'tpuid', 'RegId', 'auth_id', 'tldId'

### A this point `id_keys` contains the data organized by key. Now let's transpose the data to organize it by value and store it in `id_values`. 

In [7]:
def transpose(key_dict):
    to_return = {}
    
    for id_key in key_dict:
        instances = key_dict[id_key] # list of (package_title, host, value)

        for instance in instances:
            package_title = instance[0]
            host = instance[1]
            values = instance[2]

            for value in values:
                if value in to_return:
                    to_return[value].append((package_title, host, id_key))
                else:
                    to_return[value] = [(package_title, host, id_key)]
                    
    return to_return

id_values = transpose(id_keys) # id_value : (package_title, host, id_key)
all_values = transpose(all_keys)    

### Here's what some of the keys in `id_keys` look like

In [8]:
print id_values.keys()[0:100]

['-4641571834078775612', '699139005.1429922793', '1848942857', '09538c76-2642-499d-b190-17607094249c', '102725058.1430006360', '185108', 'TAb8fe981e-ec27-11e4-95c8-00163e219845', '9ff42421-6e36-47f2-8c01-ac2c986461dd', 'com.androiddevelopermx.blogspot.organos3d', '5982', 'ade40ba1a74a4eb4958227a18bea5472', 'rK6sP0LZRxsVD3BfaFE82973882087972', '352336', '82448', 'Ray1thhhB7nZMtobQqlcqHpj', '04491e90-6035-47ed-845f-54e5fec2ece6', 'de50feae-00da-4b47-8126-ebee7ec33660', '2A9D860605013B3D-40000105A0005961', '313690452672401075', '5GjYpsWnCrpsSjhkqvoPBjsjids0_A38pcRaBgpQhTccgbpoQfsqUqOftM_H1y-NvX39reUwb2bz0eF6GrBjX02GTZtfABipbQ', 'd622eb42-0e8e-e409-332e-7e25199a25e6', '17d5a35c-f597-4d68-9ede-63549a936b66', 'cbMJ79SqDWeKwB7H9YJkUy/00nwXQ0WoEFDF+aaTgCI2CztO+0AN4lTCNqSV5Ku4Z5phN4UO9Blq\nzn9MlMkXrjc1Sh1flzV9bxUazHx0ScvOmIwaBTC83KVCaD6QLXgKTgu2O1OCaQG9o6hHIX+P5sNh\nNgb+D3f2omZQswd9ikYfv5JXM1STba/7tB9GYxSeLbOJeXLNWZ2t2ct8jj0Jwebl3mJtDNOqXMCg\nL8Jd8ZL+bDkxA7lR2Ty9Ngi9rWA0\n', '5a5a8424-0d27-4529

### We're interested in which values appear most frequently in number of unique apps and who receives those values

In [9]:
# tally frequencies
id_values_counts = {}
for id_value in id_values:
    unique_package_titles = []
    
    for instance in id_values[id_value]:
        package_title = instance[0]
        
        if package_title not in unique_package_titles:
            unique_package_titles.append(package_title)
    
    id_values_counts[id_value] = len(unique_package_titles)
    
# sort by popularity
id_values_counts_flat = [(key, val) for key, val in id_values_counts.iteritems()]
top_id_values = sorted(id_values_counts_flat, key=lambda x: x[1], reverse=True)

### Here are the top values and their unique app frequencies

In [10]:
top_id_values[:50]

[('dcd27149-60cb-49d7-bd70-775e01b945a8', 398),
 ('0', 355),
 ('ec089e6192d63b18', 339),
 ('46621096', 233),
 ('1', 144),
 ('352584061481484', 117),
 ('1080', 110),
 ('3a5b818dde63e36e', 86),
 ('3', 83),
 ('5450213213286189855', 76),
 ('360', 76),
 ('false', 69),
 ('6', 63),
 ('46621096,46621027', 62),
 ('com.google.android.youtube', 62),
 ('9', 54),
 ('46621096,46621050', 54),
 ('100009118467642', 54),
 ('gmob-apps', 53),
 ('ON', 52),
 ('-1', 48),
 ('ifa:dcd27149-60cb-49d7-bd70-775e01b945a8', 48),
 ('2', 47),
 ('true', 43),
 ('WIFI', 42),
 ('ima_html5', 42),
 ('46621096,46621101', 42),
 ('null', 39),
 ('46621096,46621051', 37),
 ('3f4c353c7837a950dddae386fd8912bb4a27db5b', 36),
 ('v2.2', 36),
 ('760559420944499203', 32),
 ('lidar2', 30),
 ('2512c59bc73780cbe640025a039080bf', 28),
 ('5', 28),
 ('310260102923543', 25),
 ('cyclops', 25),
 ('bb11df61e36d2f3b543cef2a6874f05a', 23),
 ('1.0.6', 22),
 ('rrx68giz', 22),
 ('19', 21),
 ('300', 21),
 ('44', 20),
 ('100', 19),
 ('sodar', 19),
 ('3

In [11]:
for pair in top_id_values[:20]:
    print "%s, %s" % pair

dcd27149-60cb-49d7-bd70-775e01b945a8, 398
0, 355
ec089e6192d63b18, 339
46621096, 233
1, 144
352584061481484, 117
1080, 110
3a5b818dde63e36e, 86
3, 83
5450213213286189855, 76
360, 76
false, 69
6, 63
46621096,46621027, 62
com.google.android.youtube, 62
9, 54
46621096,46621050, 54
100009118467642, 54
gmob-apps, 53
ON, 52


### The cells below can be used to explore specific keys and values 

In [13]:
id_values['com.google.android.youtube']

[(u'com.zoodles.gamesplayer', 'www.googleadservices.com', 'bundleid'),
 (u'com.appxy.tinyscanfree', 'www.googleadservices.com', 'bundleid'),
 (u'com.unii.fling', 'www.googleadservices.com', 'bundleid'),
 (u'air.WatchESPN', 'www.googleadservices.com', 'bundleid'),
 (u'com.handmark.sportcaster', 'www.googleadservices.com', 'bundleid'),
 (u'es.lfp.laligatv', 'www.googleadservices.com', 'bundleid'),
 (u'com.neulion.smartphone.ufc.android',
  'www.googleadservices.com',
  'bundleid'),
 (u'com.vivsig.scorefinger', 'www.googleadservices.com', 'bundleid'),
 (u'com.ngsn.beta', 'www.googleadservices.com', 'bundleid'),
 (u'com.gmail.heagoo.appdm', 'www.googleadservices.com', 'bundleid'),
 (u'com.rndapp.mtamap', 'www.googleadservices.com', 'bundleid'),
 (u'com.disney.mdx.wdw.google', 'www.googleadservices.com', 'bundleid'),
 (u'com.akasoft.topplaces', 'www.googleadservices.com', 'bundleid'),
 (u'com.yahoo.mobile.client.android.weather',
  'www.googleadservices.com',
  'bundleid'),
 (u'com.Hypnosis

In [13]:
id_keys["bundleid"]

[(u'com.zoodles.gamesplayer',
  'www.googleadservices.com',
  ['com.google.android.youtube']),
 (u'com.appxy.tinyscanfree',
  'www.googleadservices.com',
  ['com.google.android.youtube']),
 (u'com.zoosk.zoosk', 'www.googleadservices.com', ['com.zoosk.zoosk']),
 (u'com.zoosk.zoosk', 'www.googleadservices.com', ['com.zoosk.zoosk']),
 (u'com.unii.fling',
  'www.googleadservices.com',
  ['com.google.android.youtube']),
 (u'com.itonline.anastasiadate',
  'www.googleadservices.com',
  ['com.itonline.anastasiadate']),
 (u'com.itonline.anastasiadate',
  'www.googleadservices.com',
  ['com.itonline.anastasiadate']),
 (u'com.itonline.anastasiadate',
  'www.googleadservices.com',
  ['com.itonline.anastasiadate']),
 (u'net.lovoo.android', 'www.googleadservices.com', ['net.lovoo.android']),
 (u'net.lovoo.android', 'www.googleadservices.com', ['net.lovoo.android']),
 (u'air.WatchESPN',
  'www.googleadservices.com',
  ['com.google.android.youtube']),
 (u'com.handmark.sportcaster',
  'www.googleadserv

### Notes

dcd27149-60cb-49d7-bd70-775e01b945a8 is the UDID, so it makes sense that it's frequently sent to parties

3f4c353c7837a950dddae386fd8912bb4a27db5b is sent to both:

(u'com.weather.Weather', 'pubads.g.doubleclick.net', 'rmid')
(u'com.modiface.eyecolor', 's.amazon-adsystem.com', 'sha1_udid')

At first sight, it might seem like they're sharing data, but Amazon's identifier suggests it's actually just a SHA1 of the UDID.


### Let's investigate when email was shared

In [37]:
email_incidents = all_values["haplesstester@gmail.com"]

In [36]:
len(email_incidents)

185

In [38]:
email_incidents

[(u'com.magnifis.parking', '173.255.249.124', 'spam'),
 (u'uk.co.economist', 'reconcile.economist.com', 'e'),
 (u'com.sewichi.client.panel', 'api.placed.com', 'e'),
 (u'com.sewichi.client.panel', 'api.placed.com', 'e'),
 (u'com.evernote', 'www.evernote.com', 'emails'),
 (u'com.billguard.android', 'trk.KISSmetrics.com', '_p'),
 (u'com.billguard.android', 'trk.KISSmetrics.com', '_p'),
 (u'com.billguard.android', 'trk.KISSmetrics.com', '_n'),
 (u'thecouponsapp.coupon', 'surveynotify.com', 'ae'),
 (u'com.zoemob.gpstracking', 'apis.zoemob.com', 'email'),
 (u'com.ToDoReminder.gen', 'wowmyfolio.com', 'email'),
 (u'com.hexadev.newkeypad.lock.screen', 'hirosz.futurehost.pl', 'email'),
 (u'com.etoolkit.lovecollage', 'lovephoto.us', 'email'),
 (u'com.billguard.android', 'trk.KISSmetrics.com', 'email'),
 (u'com.billguard.android', 'trk.KISSmetrics.com', 'email'),
 (u'com.wsl.noom', 'data.noom.com', 'email'),
 (u'com.showbox.showbox', 'showbox-showbox-live.appspot.com', 'email'),
 (u'com.showbox.sh